# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "output.csv"
city_df = pd.read_csv(csv_path)
city_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Norman Wells,75,CA,1616148239,85,65.2820,-126.8329,1.40,16.11
1,Raudeberg,52,NO,1616148240,86,61.9875,5.1352,44.01,16.53
2,Bluff,64,NZ,1616148069,93,-46.6000,168.3333,54.00,3.00
3,Cidreira,45,BR,1616148241,77,-30.1811,-50.2056,70.23,6.76
4,Ntcheu,31,MW,1616148242,83,-14.8203,34.6359,74.59,3.33
...,...,...,...,...,...,...,...,...,...
578,Nizhniy Odes,100,RU,1616148784,89,63.6445,54.8560,29.25,4.34
579,Koumac,100,NC,1616148785,80,-20.5667,164.2833,78.40,4.50
580,Yei,89,SS,1616148786,65,4.0950,30.6779,77.47,9.44
581,Sovetskiy,100,RU,1616148787,92,61.3614,63.5842,27.88,3.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = city_df["Humidity"]

In [4]:
# Plot Heatmap

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Filter by ideal weather conditions
ideal_weather_df = city_df.loc[(city_df["Max Temp"] < 95) & (city_df["Max Temp"] > 70) & (city_df["Wind Speed"] < 10) & (city_df["Cloudiness"] == 0) & (city_df["Humidity"] <= 70), :]
#ideal_weather_df

In [6]:
# Drop NaN
ideal_weather_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,Puerto Escondido,0,MX,1616148244,66,15.8500,-97.0667,74.75,4.18
45,Maţāy,0,EG,1616148272,23,28.4190,30.7792,76.15,6.20
112,Port Blair,0,IN,1616148321,56,11.6667,92.7500,86.16,3.71
152,Tombouctou,0,ML,1616148349,10,20.0000,-3.0000,83.91,7.94
193,Conakry,0,GN,1616148197,63,9.5380,-13.6773,78.30,5.73
428,Tessalit,0,ML,1616148552,10,20.1986,1.0114,83.26,7.34
449,Arlit,0,NE,1616148567,7,18.7369,7.3853,94.21,4.45
460,Gwadar,0,PK,1616148574,51,25.1216,62.3254,82.06,9.19
461,Ngozi,0,BI,1616148575,64,-2.9075,29.8306,72.84,4.29
479,Adrar,0,MR,1616148589,9,20.5022,-10.0711,80.37,8.77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Convert the dictionary into DataFrame 
hotel_df = pd.DataFrame(ideal_weather_df) 
  

# set up additional column for Hotel Name
hotel_df['Hotel Name'] = ""
  
# Observe the result 
#hotel_df

In [8]:
# Drop NaN
hotel_df.dropna()
hotel_df.reset_index()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,6,Puerto Escondido,0,MX,1616148244,66,15.8500,-97.0667,74.75,4.18,
1,45,Maţāy,0,EG,1616148272,23,28.4190,30.7792,76.15,6.20,
2,112,Port Blair,0,IN,1616148321,56,11.6667,92.7500,86.16,3.71,
3,152,Tombouctou,0,ML,1616148349,10,20.0000,-3.0000,83.91,7.94,
4,193,Conakry,0,GN,1616148197,63,9.5380,-13.6773,78.30,5.73,
5,428,Tessalit,0,ML,1616148552,10,20.1986,1.0114,83.26,7.34,
6,449,Arlit,0,NE,1616148567,7,18.7369,7.3853,94.21,4.45,
7,460,Gwadar,0,PK,1616148574,51,25.1216,62.3254,82.06,9.19,
8,461,Ngozi,0,BI,1616148575,64,-2.9075,29.8306,72.84,4.29,
9,479,Adrar,0,MR,1616148589,9,20.5022,-10.0711,80.37,8.77,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
       # print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]



In [11]:
# Create heat layer
figure_layout = {'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
hotel_layer = gmaps.marker_layer(
    marker_locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)


# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…